
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
from pyspark.sql.functions import *
import datetime as dt
file_location_zoom_bookings = "/FileStore/zoom_car_booking_data/"
file_location_zoom_customers = "/FileStore/zoom_car_customer_data/"
file_type = "json"


zoom_car_booking_data = spark.read.json(file_location_zoom_bookings)
zoom_car_customer_data = spark.read.json(file_location_zoom_customers)

zoom_car_booking_data.printSchema()
zoom_car_customer_data.printSchema()

# # The code to read json file for current date , the code is commented as we do not have a regulary scheduled pipeline
# file_location_zoom_bookings = "/FileStore/zoom_car_booking_data/"
# file_location_zoom_customers = "/FileStore/zoom_car_customer_data/"

# current_dt = spark.sql("select current_date() as current_date")
# daily_file_path_zoom_bookings= file_location_zoom_bookings + "zoom_car_bookings_" + current_dt.collect()[0][0].strftime("%Y%m%d") +".json"
# daily_file_path_zoom_customers = file_location_zoom_customers + "zoom_car_customers_" + current_dt.collect()[0][0].strftime("%Y%m%d") +".json"

# zoom_car_booking_data = spark.read.json(daily_file_path_zoom_bookings)
# zoom_car_customer_data = spark.read.json(daily_file_path_zoom_customers)




root
 |-- booking_date: string (nullable = true)
 |-- booking_id: string (nullable = true)
 |-- car_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_time: string (nullable = true)
 |-- status: string (nullable = true)
 |-- total_amount: double (nullable = true)

root
 |-- customer_id: string (nullable = true)
 |-- email: string (nullable = true)
 |-- name: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- signup_date: string (nullable = true)
 |-- status: string (nullable = true)



In [0]:
# Remove records with null values in critical fields (booking_id,customer_id, car_id, booking_date).
zoom_car_cleaned_booking_data = zoom_car_booking_data.dropna(subset=["booking_id","customer_id","car_id","booking_date"])
zoom_car_cleaned_booking_data.printSchema()

root
 |-- booking_date: string (nullable = true)
 |-- booking_id: string (nullable = true)
 |-- car_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_time: string (nullable = true)
 |-- status: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [0]:
#validate the date formats 
zoom_car_date_refactored = zoom_car_cleaned_booking_data.select(col('end_time').cast('timestamp')\
    ,col('start_time').cast('timestamp')\
        ,'booking_date'\
            ,'booking_id'\
                ,'car_id'\
                    ,'customer_id'\
                        ,'status'\
                            ,'total_amount')


# Find if all the status are as per the defined status
# There can be multiple approaches but the one i have taken is to create a small daaframe of all valid status and doing a left join from the bookings data to smaller dataset and filter out records which do not have a match 

data = [{'valid_status':'completed'},
        {'valid_status':'pending'},
        {'valid_status':'inactive'},
        {'valid_status':'active'},
        {'valid_status':'cancelled'}
        ]

defined_status = spark.createDataFrame(data)
defined_status.show(10)

bookings_joined_defined_status = zoom_car_date_refactored.join(defined_status, \
    on=zoom_car_date_refactored['status']==defined_status['valid_status']\
    ,how='left')\
        .filter(col('valid_status').isNull())

bookings_joined_defined_status.show(10)

# if the bookings_joined_defined_status returns any records then we can put it in an error file 
bookings_joined_defined_status.write.mode("overwrite").csv("/FileStore/error_file_booking_data.csv",header=True)











+------------+
|valid_status|
+------------+
|   completed|
|     pending|
|    inactive|
|      active|
|   cancelled|
+------------+

+--------+----------+------------+----------+------+-----------+------+------------+------------+
|end_time|start_time|booking_date|booking_id|car_id|customer_id|status|total_amount|valid_status|
+--------+----------+------------+----------+------+-----------+------+------------+------------+
+--------+----------+------------+----------+------+-----------+------+------------+------------+



In [0]:
%sql
--load cleaned data into staging bookings delta table 
--create the schema if not eixsts 

CREATE SCHEMA IF NOT EXISTS dev_databricks_projects.zoom_data;



In [0]:
# Load cleaned data into a staging table

bookings_joined_defined_status = zoom_car_date_refactored.join(defined_status, \
    on=zoom_car_date_refactored['status']==defined_status['valid_status']\
    ,how='left')\
        .filter(col('valid_status').isNotNull())

bookings_joined_defined_status.write.mode("overwrite").saveAsTable("dev_databricks_projects.zoom_data.staging_booking_data")



In [0]:
# application of transformations
#Parse start and end time to separate date time columns
    

bookings_joined_defined_status_new_columns = bookings_joined_defined_status.withColumn('start_date',to_date(col('start_time')))\
    .withColumn('start_timestamp',date_format(col('start_time'),'hh:mm:ss'))\
        .withColumn('end_date',to_date(col('end_time')))\
            .withColumn('end_timestamp',date_format(col('end_time'),'hh:mm:ss'))                

bookings_joined_defined_status_new_columns.select('end_date','end_timestamp','start_date','start_timestamp').show(10)

# calculate total duration of each booking

total_duration_of_booking = bookings_joined_defined_status_new_columns.withColumn('booking_duration',(unix_timestamp('end_time')-unix_timestamp('start_time'))/60)

total_duration_of_booking.select('booking_duration').show(10)




+----------+-------------+----------+---------------+
|  end_date|end_timestamp|start_date|start_timestamp|
+----------+-------------+----------+---------------+
|2024-08-05|     05:41:00|2024-08-05|       05:40:00|
|2024-08-05|     05:41:00|2024-08-05|       05:40:00|
|2024-08-05|     12:23:00|2024-08-05|       12:20:00|
|2024-08-05|     12:56:00|2024-08-05|       12:50:00|
|2024-08-05|     04:12:00|2024-08-05|       04:10:00|
|2024-08-05|     12:51:00|2024-08-05|       12:50:00|
|2024-08-05|     04:13:00|2024-08-05|       04:10:00|
|2024-08-05|     02:15:00|2024-08-05|       02:10:00|
|2024-08-05|     12:13:00|2024-08-05|       12:10:00|
|2024-08-05|     04:16:00|2024-08-05|       04:10:00|
+----------+-------------+----------+---------------+
only showing top 10 rows

+----------------+
|booking_duration|
+----------------+
|            15.0|
|             3.0|
|             6.0|
|             3.0|
|             5.0|
|             1.0|
|             5.0|
|             3.0|
|        